In [ ]:
# Cell 1: Installation des dépendances
!pip install stable-baselines3 gymnasium numba numpy pandas matplotlib --quiet
print("✅ Dependencies installed")

In [ ]:
# Cell 2: Clone du repo et setup
import os
import sys

# Clone if needed
if not os.path.exists('/content/Code-traffic-flow'):
    !git clone https://github.com/elonmj/Code-traffic-flow.git
    print("✅ Repository cloned")
else:
    # Pull latest changes
    %cd /content/Code-traffic-flow
    !git pull origin main
    print("✅ Repository updated")

%cd /content/Code-traffic-flow
sys.path.insert(0, '/content/Code-traffic-flow')
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Cell 3: Vérification GPU
import torch
from numba import cuda

print("=" * 50)
print("GPU CHECK")
print("=" * 50)
print(f"PyTorch CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  Device: {torch.cuda.get_device_name(0)}")
print(f"Numba CUDA: {cuda.is_available()}")
if cuda.is_available():
    print(f"  Device: {cuda.get_current_device().name}")
print("=" * 50)

In [ ]:
# Cell 4: Imports
import json
import time
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import BaseCallback

from Code_RL.src.env.traffic_signal_env_direct_v3 import TrafficSignalEnvDirectV3
from arz_model.config import create_victoria_island_config

print("✅ All imports successful")

In [ ]:
# Cell 5: Configuration du scénario congestionné
# ==========================================
# PARAMÈTRES CLÉS - SCÉNARIO CONGESTIONNÉ
# ==========================================

TRAINING_TIMESTEPS = 5000  # 5k steps comme demandé
EVAL_EPISODES = 3

# Densités élevées pour créer de la congestion
DEFAULT_DENSITY = 120.0  # veh/km (était 80)
INFLOW_DENSITY = 180.0   # veh/km (était 100)

# Reward weights optimisés
REWARD_WEIGHTS = {
    'alpha': 5.0,   # Pénalité densité (était 1.0)
    'kappa': 0.3,   # Pénalité switch (était 0.1)
    'mu': 0.1       # Récompense débit (était 0.5)
}

print("=" * 50)
print("SCÉNARIO CONGESTIONNÉ (LOS D - Rush Hour)")
print("=" * 50)
print(f"Densité initiale: {DEFAULT_DENSITY} veh/km")
print(f"Densité d'entrée: {INFLOW_DENSITY} veh/km")
print(f"Reward weights: {REWARD_WEIGHTS}")
print(f"Training timesteps: {TRAINING_TIMESTEPS}")
print("=" * 50)

In [ ]:
# Cell 6: Fonction de création d'environnement
def create_congested_env(quiet=True):
    """Crée l'environnement avec scénario congestionné."""
    arz_config = create_victoria_island_config(
        t_final=450.0,
        output_dt=15.0,
        cells_per_100m=4,
        default_density=DEFAULT_DENSITY,
        inflow_density=INFLOW_DENSITY,
        use_cache=False
    )
    
    arz_config.rl_metadata = {
        'observation_segment_ids': [s.id for s in arz_config.segments],
        'decision_interval': 15.0,
    }
    
    class SimpleConfig:
        def __init__(self, arz_config):
            self.arz_simulation_config = arz_config
            self.rl_env_params = {
                'dt_decision': 15.0,
                'observation_segment_ids': None,
                'reward_weights': REWARD_WEIGHTS
            }
    
    rl_config = SimpleConfig(arz_config)
    
    env = TrafficSignalEnvDirectV3(
        simulation_config=rl_config.arz_simulation_config,
        decision_interval=rl_config.rl_env_params.get('dt_decision', 15.0),
        observation_segment_ids=rl_config.rl_env_params.get('observation_segment_ids'),
        reward_weights=rl_config.rl_env_params.get('reward_weights'),
        quiet=quiet
    )
    return env

print("✅ Environment factory ready")

In [ ]:
# Cell 7: Callback pour tracking
class TrainingCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.episode_rewards = []
        self.episode_lengths = []
        self.current_ep_reward = 0.0
        self.current_ep_length = 0
        
    def _on_step(self) -> bool:
        self.current_ep_reward += self.locals['rewards'][0]
        self.current_ep_length += 1
        
        if self.locals['dones'][0]:
            self.episode_rewards.append(self.current_ep_reward)
            self.episode_lengths.append(self.current_ep_length)
            self.current_ep_reward = 0.0
            self.current_ep_length = 0
            
            if len(self.episode_rewards) % 10 == 0:
                avg_last_10 = np.mean(self.episode_rewards[-10:])
                print(f"  Episode {len(self.episode_rewards)}: Avg reward (last 10) = {avg_last_10:.2f}")
        
        return True

print("✅ Callback ready")

In [ ]:
# Cell 8: Fonction d'évaluation
def evaluate_policy(env, policy_type='fixed_time', model=None, n_episodes=3, fixed_time_interval=30.0):
    """Évalue une politique sur plusieurs épisodes."""
    metrics = {'rewards': [], 'densities': [], 'throughputs': []}
    
    for ep in range(n_episodes):
        obs, info = env.reset()
        done = truncated = False
        ep_reward = 0.0
        ep_densities = []
        ep_throughputs = []
        time_since_switch = 0.0
        current_action = 0
        
        while not (done or truncated):
            if policy_type == 'model':
                action, _ = model.predict(obs, deterministic=True)
            else:  # fixed_time
                time_since_switch += env.decision_interval
                if time_since_switch >= fixed_time_interval:
                    action = 1  # Switch
                    time_since_switch = 0.0
                else:
                    action = 0  # Keep
            
            obs, reward, done, truncated, info = env.step(action)
            ep_reward += reward
            
            if 'avg_density' in info:
                ep_densities.append(info['avg_density'])
            if 'throughput' in info:
                ep_throughputs.append(info['throughput'])
                
        metrics['rewards'].append(ep_reward)
        metrics['densities'].append(np.mean(ep_densities) if ep_densities else 0.0)
        metrics['throughputs'].append(np.sum(ep_throughputs) if ep_throughputs else 0.0)
        
        print(f"  Episode {ep+1}: Reward={ep_reward:.2f}, Density={metrics['densities'][-1]:.4f}")
    
    return {
        'mean_reward': float(np.mean(metrics['rewards'])),
        'std_reward': float(np.std(metrics['rewards'])),
        'mean_density': float(np.mean(metrics['densities'])),
        'mean_throughput': float(np.mean(metrics['throughputs'])),
        'all_rewards': [float(r) for r in metrics['rewards']]
    }

print("✅ Evaluation function ready")

---
## 📊 Part 1: Baseline Evaluation

In [ ]:
# Cell 9: Évaluation Baseline
print("=" * 60)
print("PART 1: BASELINE (Fixed-Time 30s)")
print("=" * 60)

env = create_congested_env(quiet=True)
baseline_results = evaluate_policy(env, policy_type='fixed_time', n_episodes=EVAL_EPISODES)
env.close()

print(f"\n📊 Baseline Results:")
print(f"   Mean Reward: {baseline_results['mean_reward']:.2f} ± {baseline_results['std_reward']:.2f}")
print(f"   Mean Density: {baseline_results['mean_density']:.4f}")
print(f"   Mean Throughput: {baseline_results['mean_throughput']:.2f}")

---
## 🎯 Part 2: DQN Training (5000 steps)

In [ ]:
# Cell 10: DQN Training
print("=" * 60)
print("PART 2: DQN TRAINING")
print("=" * 60)

env = create_congested_env(quiet=True)

model = DQN(
    "MlpPolicy",
    env,
    learning_rate=1e-4,
    buffer_size=10000,
    learning_starts=500,
    batch_size=64,
    gamma=0.99,
    target_update_interval=250,
    exploration_fraction=0.3,
    exploration_final_eps=0.05,
    verbose=1
)

callback = TrainingCallback()

print(f"\n🚀 Starting training for {TRAINING_TIMESTEPS} timesteps...")
start_time = time.time()

model.learn(total_timesteps=TRAINING_TIMESTEPS, callback=callback)

elapsed = time.time() - start_time
print(f"\n✅ Training completed in {elapsed/60:.1f} minutes")
print(f"   Total episodes: {len(callback.episode_rewards)}")
print(f"   Mean episode reward: {np.mean(callback.episode_rewards):.2f}")
print(f"   Max episode reward: {np.max(callback.episode_rewards):.2f}")

---
## 📈 Part 3: RL Evaluation

In [ ]:
# Cell 11: RL Evaluation
print("=" * 60)
print("PART 3: RL EVALUATION")
print("=" * 60)

rl_results = evaluate_policy(env, model=model, policy_type='model', n_episodes=EVAL_EPISODES)
env.close()

print(f"\n📊 RL Results:")
print(f"   Mean Reward: {rl_results['mean_reward']:.2f} ± {rl_results['std_reward']:.2f}")
print(f"   Mean Density: {rl_results['mean_density']:.4f}")
print(f"   Mean Throughput: {rl_results['mean_throughput']:.2f}")

---
## 🏆 Part 4: Final Comparison

In [ ]:
# Cell 12: Comparaison finale
print("=" * 60)
print("FINAL COMPARISON: BASELINE vs RL")
print("=" * 60)

# Calcul des améliorations
improvement = {
    'reward': (rl_results['mean_reward'] - baseline_results['mean_reward']) / abs(baseline_results['mean_reward']) * 100,
    'density': (rl_results['mean_density'] - baseline_results['mean_density']) / baseline_results['mean_density'] * 100,
    'throughput': (rl_results['mean_throughput'] - baseline_results['mean_throughput']) / baseline_results['mean_throughput'] * 100 if baseline_results['mean_throughput'] > 0 else 0
}

print(f"\n{'Métrique':<20} {'Baseline':<15} {'RL (DQN)':<15} {'Gain':<10}")
print("-" * 60)
print(f"{'Reward':<20} {baseline_results['mean_reward']:<15.2f} {rl_results['mean_reward']:<15.2f} {improvement['reward']:+.2f}%")
print(f"{'Densité (veh/m)':<20} {baseline_results['mean_density']:<15.4f} {rl_results['mean_density']:<15.4f} {improvement['density']:+.2f}%")
print(f"{'Débit (veh/s)':<20} {baseline_results['mean_throughput']:<15.2f} {rl_results['mean_throughput']:<15.2f} {improvement['throughput']:+.2f}%")

print("\n" + "=" * 60)
if improvement['reward'] > 0:
    print(f"🎉 L'agent RL a amélioré le reward de {improvement['reward']:+.2f}%!")
else:
    print(f"⚠️  L'agent RL a un reward inférieur de {abs(improvement['reward']):.2f}%")
print("=" * 60)

In [ ]:
# Cell 13: Génération des figures
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Figure 1: Courbe d'apprentissage
ax1 = axes[0]
episodes = range(1, len(callback.episode_rewards) + 1)
ax1.plot(episodes, callback.episode_rewards, 'b-', alpha=0.3, label='Episode reward')

# Moving average
window = min(10, len(callback.episode_rewards))
if len(callback.episode_rewards) >= window:
    moving_avg = np.convolve(callback.episode_rewards, np.ones(window)/window, mode='valid')
    ax1.plot(range(window, len(callback.episode_rewards)+1), moving_avg, 'r-', linewidth=2, label=f'Moving avg ({window} ep)')

ax1.axhline(y=baseline_results['mean_reward'], color='green', linestyle='--', linewidth=2, label='Baseline')
ax1.set_xlabel('Episode', fontsize=12)
ax1.set_ylabel('Reward', fontsize=12)
ax1.set_title('Courbe d\'apprentissage DQN (Scénario Congestionné)', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Figure 2: Comparaison
ax2 = axes[1]
metrics = ['Reward', 'Densité\n(×1000)', 'Débit\n(×100)']
baseline_vals = [baseline_results['mean_reward'], baseline_results['mean_density']*1000, baseline_results['mean_throughput']/100]
rl_vals = [rl_results['mean_reward'], rl_results['mean_density']*1000, rl_results['mean_throughput']/100]

x = np.arange(len(metrics))
width = 0.35

bars1 = ax2.bar(x - width/2, baseline_vals, width, label='Baseline (Fixed-Time)', color='steelblue')
bars2 = ax2.bar(x + width/2, rl_vals, width, label='RL (DQN)', color='coral')

ax2.set_ylabel('Valeur', fontsize=12)
ax2.set_title('Comparaison Baseline vs RL', fontsize=14)
ax2.set_xticks(x)
ax2.set_xticklabels(metrics)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Add improvement annotations
for i, (b, r) in enumerate(zip(baseline_vals, rl_vals)):
    if i == 0:  # Reward
        pct = improvement['reward']
    elif i == 1:  # Density
        pct = improvement['density']
    else:  # Throughput
        pct = improvement['throughput']
    ax2.annotate(f'{pct:+.1f}%', xy=(x[i] + width/2, r), ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('/content/thesis_stage2_v2_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Figure saved to /content/thesis_stage2_v2_results.png")

In [ ]:
# Cell 14: Sauvegarde des résultats JSON
results = {
    'config': {
        'default_density': DEFAULT_DENSITY,
        'inflow_density': INFLOW_DENSITY,
        'reward_weights': REWARD_WEIGHTS,
        'training_timesteps': TRAINING_TIMESTEPS,
        'eval_episodes': EVAL_EPISODES
    },
    'baseline': baseline_results,
    'rl_dqn': rl_results,
    'improvement': improvement,
    'training': {
        'total_episodes': len(callback.episode_rewards),
        'mean_reward': float(np.mean(callback.episode_rewards)),
        'max_reward': float(np.max(callback.episode_rewards)),
        'min_reward': float(np.min(callback.episode_rewards)),
        'episode_rewards': [float(r) for r in callback.episode_rewards]
    }
}

with open('/content/thesis_stage2_v2_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✅ Results saved to /content/thesis_stage2_v2_results.json")
print("\n📋 Summary JSON:")
print(json.dumps({
    'baseline_reward': baseline_results['mean_reward'],
    'rl_reward': rl_results['mean_reward'],
    'improvement_pct': improvement['reward']
}, indent=2))

In [ ]:
# Cell 15: Download les fichiers
from google.colab import files

print("📥 Downloading results...")
files.download('/content/thesis_stage2_v2_results.png')
files.download('/content/thesis_stage2_v2_results.json')
print("\n✅ Download complete!")